## SlideRule - ICESat2 Jupyter Client

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()

# Default Matplotlib Plot of Dataframe Column 
def dfplot(df, col, cond_col="", cond_val=0):
    if cond_col != "":
        c = df[df[cond_col] == cond_val]
        s = c[pd.isnull(c[col]) == False][col]
    else:
        s = df[pd.isnull(df[col]) == False][col]
    s.plot(figsize=(15,5))
    plt.title(col)
    plt.show()

# Sum of Dataframe Column
def dfsum(df, col, cond_col="", cond_val=0):
    if cond_col != "":
        total = df[df[cond_col] == cond_val][col].sum()
    else:
        total = df[col].sum()
    return total

# Matplotlib Histogram of Value Counts in Dataframe Column
def dfplotvc(df, col, cond_col="", cond_val=0):
    if cond_col != "":
        vc = df[df[cond_col] == cond_val][col].value_counts()
    else:
        vc = df[col].value_counts()
    plt.clf()
    plt.figure(figsize=(15,8))
    vc.plot(kind='bar')
    plt.show()  

# Interactive Bokeh Plot of Dataframe Column
# ... note that x axis is a recreated list of item number
# ... and does not reflect original index in dataframe
def dfbokeh(df, col, cond_col="", cond_val=0):
    p = figure(title=col, plot_height=300, plot_width=600)
    if cond_col != "":
        c = df[df[cond_col] == cond_val]
        s = c[pd.isnull(c[col]) == False][col]
    else:
        s = df[pd.isnull(df[col]) == False][col]
    y = s.values
    x = [i for i in range(len(y))]
    r = p.line(x, y, color="#2222aa", line_width=3)
    show(p, notebook_handle=True)


Loading BokehJS ...

In [2]:
import requests
import json

Run `sliderule plugins/icesat2/server.lua` prior to executing any cell after this

In [3]:
server_url = 'http://127.0.0.1:9081'
def engine (api, parm):
    url = '%s/engine/%s' % (server_url, api)
    return requests.post(url, data=parm, stream=True)

In [4]:
filename = "/data/ATLAS/ATL03_20200304065203_10470605_003_01.h5"
dataset = "gt2l/heights/dist_ph_along"
datatag = 0

rqst_dict = {
        "filename": filename,
        "dataset": dataset,
        "id": datatag
    }

p = json.dumps(rqst_dict)

print(p)

{"filename": "/data/ATLAS/ATL03_20200304065203_10470605_003_01.h5", "dataset": "gt2l/heights/dist_ph_along", "id": 0}


In [5]:
d = engine("h5", p)
for line in d.iter_lines():
    num_elem = int(len(line) / 4)
    val_array = np.frombuffer(line, dtype=np.float32, count=num_elem)
    df = pd.DataFrame(data=val_array, index=[i for i in range(num_elem)], columns=[dataset])
    break

In [6]:
df.head()

,gt2l/heights/dist_ph_along
0,0.106370
1,0.106865
2,0.106748
3,0.106908
4,0.814255


In [7]:
df.tail()

,gt2l/heights/dist_ph_along
113,16.160875
114,16.867819
115,17.574907
116,17.574154
117,18.281281


In [8]:
df.describe()

,gt2l/heights/dist_ph_along
count,118.000000
mean,9.466295
std,5.723091
min,0.106370
25%,4.851367
50%,9.551124
75%,14.747703
max,19.909487


In [9]:
# Display a slice of dataframe
df[1:5]

,gt2l/heights/dist_ph_along
1,0.106865
2,0.106748
3,0.106908
4,0.814255


In [10]:
# Display the sum of signal strength across the entire dataframe
print(dfsum(df, dataset))

1117.0228


In [11]:
# Create an interactive plot of a column
dfbokeh(df, dataset)